# <center>Tabular Playground Series - June/2021<center>
## <center>Keras Neural Network with Skip Connections<center>
---

- Neural Network inspired by [@pourchot's](https://www.kaggle.com/pourchot) notebook [Simple Neural Network](https://www.kaggle.com/pourchot/simple-neural-network).
- Using KNN features provided by [@melanie7744's](https://www.kaggle.com/melanie7744) notebook [TPS6-Boost your score with KNN features](https://www.kaggle.com/melanie7744/tps6-boost-your-score-with-knn-features).

    
<br>
<br>    
    
![NN](https://i.imgur.com/4up4HK2.png)
 <center>Simplified Representation of the Neural Network<center>
<br>


My other notebooks in this competition:
- [Tabular Playground Series - June/2021: Starter - EDA + Base LightGBM](https://www.kaggle.com/jonaspalucibarbosa/tps06-21-starter-eda-base-lgbm)
- [Tabular Playground Series - June/2021: Simple Neural Network with Keras](https://www.kaggle.com/jonaspalucibarbosa/tps06-21-simple-nn-with-keras)
- [Tabular Playground Series - June/2021: Keras Neural Network with Embedding Layer](https://www.kaggle.com/jonaspalucibarbosa/tps06-21-keras-nn-with-embedding)
- [Tabular Playground Series - June/2021: Wide and Deep Neural Network with Keras](https://www.kaggle.com/jonaspalucibarbosa/tps06-21-wide-and-deep-nn-w-keras)
- [Tabular Playground Series - June/2021: LightAutoML with KNN Features](https://www.kaggle.com/jonaspalucibarbosa/tps06-21-lightautoml-w-knn-feats)

## Importing Libraries and Datasets

In [ ]:
import pandas as pd       
import matplotlib as mat
import matplotlib.pyplot as plt    
import numpy as np
import seaborn as sns
%matplotlib inline

import random
import os

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import callbacks
import tensorflow_addons as tfa
from keras.models import Model
from keras import backend as K

In [ ]:
#Reproducible results
from numpy.random import seed
seed(42)
from tensorflow.random import set_seed
set_seed(42)

random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv', index_col = 'id')
Y_train = df_train['target'].copy()
X_train = df_train.copy().drop('target', axis = 1)

X_test = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv', index_col = 'id')

In [ ]:
train_knn = np.load("../input/tps6-boost-your-score-with-knn-features/add_feat_train.npy")
test_knn = np.load("../input/tps6-boost-your-score-with-knn-features/add_feat_test.npy")

scaler = MinMaxScaler()
train_knn = pd.DataFrame(scaler.fit_transform(train_knn))
test_knn = pd.DataFrame(scaler.transform(test_knn),index = range (200000,300000,1))

train_knn.columns = [('knn_{0:d}').format(i) for i in range(1,10)]
test_knn.columns = [('knn_{0:d}').format(i) for i in range(1,10)]

In [ ]:
X_train = pd.concat([X_train, train_knn], axis=1)
X_test = pd.concat([X_test, test_knn], axis=1)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
class_map = {'Class_1': 0,
            'Class_2': 1,
            'Class_3': 2,
            'Class_4': 3,
            'Class_5': 4,
            'Class_6': 5,
            'Class_7': 6,
            'Class_8': 7,
            'Class_9': 8}
Y_train = Y_train.map(class_map).astype('int')
Y_train

In [ ]:
#Converting target series to matrix for multiclass classification on Keras

Y_train = to_categorical(Y_train)
Y_train

## Creating and Evaluating the NN

In [ ]:
cce = keras.losses.CategoricalCrossentropy()

def custom_metric(y_true, y_pred):
    y_pred = K.clip(y_pred, 1e-15, 1-1e-15)
    loss = K.mean(cce(y_true, y_pred))
    return loss

early_stopping = callbacks.EarlyStopping(
    monitor='val_custom_metric',
    patience=10,
    min_delta=0.0000001,
    restore_best_weights=True,
)

#New callback
plateau = callbacks.ReduceLROnPlateau(
    monitor='val_custom_metric',
    factor = 0.5,                                     
    patience = 2,                                   
    min_delt = 0.0000001,                                
    cooldown = 0,                               
    verbose = 1
) 

In [ ]:
def res_model():

    inputs = layers.Input(shape = (75,))
    knn_fts = layers.Input(shape = (9,))
    
    #Embedding + Convolution
    embed = layers.Embedding(360, 8, embeddings_regularizer='l2')(inputs)
    embed = layers.Conv1D(12, kernel_size=1, activation='relu')(embed)
    embed = layers.Flatten()(embed)
    
    knn = layers.BatchNormalization()(knn_fts)
    
    #Residual Blocks Layers
    x1 = layers.Dropout(0.3)(layers.Concatenate()([embed, knn]))
    x1 = layers.Dense(units = 32, activation = 'relu', kernel_initializer = "he_normal")(x1)
    x1 = layers.BatchNormalization()(x1) 
    x2 = layers.Dropout(0.4) (layers.Concatenate()([embed, x1]))
    x2 = layers.Dense(units = 32, activation = 'relu', kernel_initializer = "he_normal")(x2)
    x2 = layers.BatchNormalization()(x2)
    x3 = layers.Dropout(0.3) (layers.Concatenate()([embed, x1, x2]))
    x3 = layers.Dense(units = 16, activation = 'relu', kernel_initializer = "he_normal")(x3)
    x3 = layers.BatchNormalization()(x3)
    x4 = layers.Dropout(0.3) (layers.Concatenate()([embed, x1, x2 ,x3]))
    x4 = layers.Dense(units = 16, activation = 'relu', kernel_initializer = "he_normal")(x4)
    x4 = layers.BatchNormalization()(x4)
    
    #Final Layer (Output)
    outputs = layers.Dense(9, activation = 'softmax', kernel_initializer = "lecun_normal")(x4)
    
    model = keras.Model(inputs=[inputs, knn_fts], outputs=outputs)
    
    return model

In [ ]:
keras.backend.clear_session()

model = res_model()
model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(learning_rate=0.0002), metrics=custom_metric)

model.summary()

In [ ]:
X_train_split, X_val_split, Y_train_split, Y_val_split = train_test_split(X_train, Y_train, test_size = 0.2, random_state = 42
                                                    , stratify = Y_train)

In [ ]:
history = model.fit([X_train_split.iloc[:, :75], X_train_split.iloc[:, 75:]], Y_train_split,
          batch_size = 256, epochs = 100,
          validation_data=([X_val_split.iloc[:, :75], X_val_split.iloc[:, 75:]], Y_val_split),                    
          callbacks=[early_stopping, plateau]);

In [ ]:
score = model.evaluate([X_val_split.iloc[:, :75], X_val_split.iloc[:, 75:]], Y_val_split, verbose = 0)
print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}%'.format(score[1] * 100))

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))
sns.lineplot(x = history.epoch, y = history.history['loss'])
sns.lineplot(x = history.epoch, y = history.history['val_loss'])
ax.set_title('Learning Curve (Loss)')
ax.set_ylabel('Loss')
ax.set_xlabel('Epoch')
ax.legend(['train', 'test'], loc='best')
plt.show()

## Making Predictions

In [ ]:
Y_train = df_train['target'].copy()
Y_train = Y_train.map(class_map).astype('int')
Y_train

In [ ]:
def prediction (X_train, Y_train, X_test):
    
    keras.backend.clear_session()

    kfold = StratifiedKFold(n_splits = 25)

    y_pred = np.zeros((100000,9))
    train_oof = np.zeros((200000,9))
    
    for idx in kfold.split(X=X_train, y=Y_train):
        train_idx, val_idx = idx[0], idx[1]
        xtrain = X_train.iloc[train_idx]
        ytrain = Y_train.iloc[train_idx]
        xval = X_train.iloc[val_idx]
        yval = Y_train.iloc[val_idx]
        
        ytrain = to_categorical(ytrain)
        yval = to_categorical(yval)
        
        # fit model for current fold
        model = res_model()
        model.compile(loss='categorical_crossentropy'
                      , optimizer = keras.optimizers.Adam(learning_rate=0.0002), metrics=custom_metric)
        
        model.fit([xtrain.iloc[:, :75], xtrain.iloc[:, 75:]], ytrain,       
        batch_size = 256, epochs = 100,
        validation_data=([xval.iloc[:, :75], xval.iloc[:, 75:]], yval),  
        callbacks=[early_stopping, plateau]);

        #create predictions
        y_pred += model.predict([X_test.iloc[:, :75], X_test.iloc[:, 75:]])/kfold.n_splits
        print(y_pred)
               
        val_pred = model.predict([xval.iloc[:, :75], xval.iloc[:, 75:]])
        # getting out-of-fold predictions on training set
        train_oof[val_idx] = val_pred
        
        # calculate and append logloss
        fold_logloss = metrics.log_loss(yval,val_pred)
        print("Logloss: {0:0.5f}". format(fold_logloss))
  
    return y_pred, train_oof

In [ ]:
nn_pred, train_oof = prediction (X_train, Y_train, X_test)

In [ ]:
print("Logloss: {0:0.6f}".format(metrics.log_loss(Y_train,train_oof)))

In [ ]:
train_oof = pd.DataFrame(train_oof, columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9'])
train_oof

In [ ]:
pred_test = pd.DataFrame(nn_pred, columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9'])
pred_test

In [ ]:
train_oof.to_csv('nn_train_oof.csv', index=False)
train_oof

In [ ]:
output = pred_test
output['id'] = X_test.index
output.to_csv('submission.csv', index=False)

output